In [32]:
# initialize project:
using Pkg;
Pkg.activate("./MyProject");

# Find current path
CURRENT = pwd();
LIB_PATH = joinpath(CURRENT, "libs");

# include auxiliary Julia scripts:
for file in readdir(LIB_PATH)
    # Check if the file ends with ".jl" extension
    if endswith(file, ".jl")
        include(joinpath(LIB_PATH, file))
    end
end

using Polymake
const pm = Polymake;
const g = GAP.Globals;

  Activating project at `~/Documents/GitHub/StabTheory/MyProject`


In [34]:
i3 = canonical_maximal_isotropic(3)
symplectic_orbit = SympOrbit(3, SympPerm(3), Set(i3)).Orbit

I3loc = local_isotropics(symplectic_orbit, 3)
A3loc = stabilizer_coefficients(3, I3loc)

216×64 Matrix{Int64}:
 1   1  0   0   0   0  0   0   1   1  …   0  0  0   1   1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0  -1   1      0  0  0   1  -1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0   1  -1      0  0  0   1  -1  0  0  0  0  0  0
 1   1  0   0   0   0  0   0  -1  -1      0  0  0   1   1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0   1  -1      0  0  0  -1   1  0  0  0  0  0  0
 1   1  0   0   0   0  0   0  -1  -1  …   0  0  0  -1  -1  0  0  0  0  0  0
 1   1  0   0   0   0  0   0   1   1      0  0  0  -1  -1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0  -1   1      0  0  0  -1   1  0  0  0  0  0  0
 1   0  0   1   1   0  0   1   0   0      0  0  0   0   0  0  0  0  0  0  0
 1   0  0  -1   1   0  0  -1   0   0      0  0  0   0   0  0  0  0  0  0  0
 ⋮                  ⋮                 ⋱         ⋮                ⋮        
 1   0  0   1   0   0  0   0  -1   0      0  0  0   0   0  0  0  0  0  0  0
 1   1  0   0   1   1  0   0   0   0      1  0  0   0   0  0  0  0 

In [39]:
all_locals = Set{Vector{Int}}(
    [
        IIX, IIY, IIZ, IXI, IYI, IZI, XII, YII, ZII
    ]
)

E3 = find_local_closure(all_locals)
det_vertices = find_all_possible_local_value_assignments(E3)
det_vertices_matrix = hcat([value_assignment_to_pauli_basis(v, 3) for v in det_vertices]...)

64×512 Matrix{Int64}:
  1   1   1   1   1   1   1   1   1  …   1   1   1   1   1   1   1   1   1
  1   1  -1  -1  -1   1  -1   1  -1      1  -1   1   1  -1   1   1  -1   1
  1   1  -1   1  -1  -1   1   1   1     -1   1  -1   1   1  -1   1  -1   1
 -1   1  -1   1   1  -1   1   1   1      1   1   1   1  -1  -1   1  -1  -1
 -1  -1   1  -1   1  -1   1   1  -1      1   1   1  -1   1   1   1   1  -1
 -1  -1  -1   1  -1  -1  -1   1   1  …   1  -1   1  -1  -1   1   1  -1  -1
 -1  -1  -1  -1  -1   1   1   1  -1     -1   1  -1  -1   1  -1   1  -1  -1
  1  -1  -1  -1   1   1   1   1  -1      1   1   1  -1  -1  -1   1  -1   1
  1   1  -1   1  -1   1   1  -1   1      1  -1  -1   1   1   1   1  -1   1
  1   1   1  -1   1   1  -1  -1  -1      1   1  -1   1  -1   1   1   1   1
  ⋮                   ⋮              ⋱           ⋮                   ⋮  
 -1  -1  -1   1   1   1  -1  -1   1  …   1  -1   1   1   1   1  -1  -1   1
 -1   1  -1  -1  -1   1  -1   1  -1      1   1  -1  -1  -1  -1  -1  -1   1
 -1  

In [35]:
find_rank_count_for_given_given_value_assignment(first(det_vertices), 3, A3loc)

63

In [21]:
ex = Vector{Float64}(value_assignment_to_pauli_basis(first(det_vertices), 3))

64-element Vector{Float64}:
  1.0
  1.0
  1.0
 -1.0
 -1.0
 -1.0
 -1.0
  1.0
  1.0
  1.0
  ⋮
 -1.0
 -1.0
 -1.0
 -1.0
  1.0
 -1.0
 -1.0
 -1.0
  1.0

In [42]:
all_cncs = generate_all_cncs(3, [0])
count = 0
for cnc in all_cncs
    if count == 5
        break
    end
    rank = find_rank_count_for_given_given_value_assignment(cnc.value_assignment, 3, A3loc)
    if rank != 63
        cnc_pauli = Vector{Float64}(value_assignment_to_pauli_basis(cnc.value_assignment, 3))
        solver = Initializer(3, cnc_pauli, det_vertices_matrix);
        println(solver.Feasible)
        println(solver.Mixture)
    end
    count += 1 
end

true
Convex
false
Affine
false
Affine
true
Convex
true
Convex
